## CV14 - 카메라 스티커앱을 개선하자
### 프로젝트 1. moviepy로 비디오 처리하기

In [1]:
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

#### moviepy를 이용하여 주피터 노트북에서 비디오를 읽고 쓰기

In [2]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# 쓰기
result_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

t:   0%|          | 0/404 [00:00<?, ?it/s, now=None]                

Moviepy - Building video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4.
MoviePy - Writing audio in mvpyresultTEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4


#### moviepy로 읽은 동영상을 numpy로 변환하여 영상밝기를 50% 어둡게 만든 후 저장하기

In [3]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# clip 에서 numpy 로 데이터 추출
vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)
for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

# 새 clip 만들기
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

# 쓰기
result_video_path2 = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult2.mp4'
outclip.write_videofile(result_video_path2, fps=30)

t:  17%|█▋        | 67/403 [00:00<00:00, 662.61it/s, now=None]

Moviepy - Building video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult2.mp4.
Moviepy - Writing video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult2.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult2.mp4


#### moivepy와 OpenCV를 사용할 때 영상을 읽고 쓰는 시간을 측정하기

In [4]:
# CASE 1 : moviepy 사용
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

mvpy_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

t:  16%|█▋        | 66/403 [00:00<00:00, 656.01it/s, now=None]

Moviepy - Building video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4.
Moviepy - Writing video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4
[INFO] moviepy time : 2.60ms


In [5]:
# CASE 2 : OpenCV 사용
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

[INFO] cv time : 1.23ms


openCV가 moivepy보다 처리 속도가 빠르다.
### 프로젝트 2. 어디까지 만들고 싶은지 정의하기
#### 웹캠을 이용한 실시간 스티커앱 만들기
addsticker.py에서 img2sticker_orig 메소드를 복사하여 새로운 newaddsticker.py에 img2sticker라는 이름으로 저장한다.  
webcam_sticker.py라는 파일에 다음 코드를 입력한다.

In [ ]:
import numpy as np
import cv2
import dlib

from newaddsticker import img2sticker

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    vc = cv2.VideoCapture(0)
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.

    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    # for i in range(vlen):
    while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전

        # 스티커 메소드를 사용
        img_result = img2sticker(img, img_sticker.copy(), detector_hog, landmark_predictor)   

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)

        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break


if __name__ == '__main__':
    main()

#### 얼굴을 인식하지 못하는 거리가 있다.
일반적으로 15cm ~ 130cm 사이의 거리에서 얼굴 인식이 된다고 한다.  
#### 고개를 상하좌우로 움직여서 몇 도까지 정상적으로 작동하는지 확인
yaw, pitch, roll각도
#### 스티커앱의 스펙(허용 거리, 허용 인원 수, 허용 각도, 안정성) 정하기
### 프로젝트 3. 스티커 Out Bound 예외처리 하기
#### 좌우 경계 밖으로 나가는 경우 예외 상황 발생
#### 코드을 확인하여 수정하기

In [ ]:
refined_x = x - w // 2
refined_y = y - h

if refined_y < 0:
    img_sticker = img_sticker[-refined_y:]
    refined_y = 0

###
# TODO : x 축 예외처리 코드 추가
###
if refined_x < 0:
    img_sticker = img_sticker[:, -refined_x:]
    refined_x = 0
elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
    img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]

img_bgr = img_orig.copy()
sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]

#### 다른 예외 상황 정의하기
예) 고개를 왼쪽아래로 향하게 할 때 스티커 모양이 일정한 것
### 프로젝트 4. 스티커앱 분석 - 거리, 인원 수, 각도, 시계열 안정성
#### 멀어지는 경우에 스티커앱이 작동하지 않는 이유는 뭘까?
detection이 안 되기 때문에, detector_hog단계에서  

In [ ]:
# preprocess
img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
# detector
img_rgb_vga = cv2.resize(img_rgb, (640, 360))
dlib_rects = detector_hog(img_rgb_vga, 0)
if len(dlib_rects) < 1:
    return img_orig

#### detection 문제 해결하기 위해 img2sticker 메소드 코드를 간단히 수정하기
이미지 피라미드를 조절하여 성능 향상

In [ ]:
def img2sticker(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

    # detector
    img_rgb_vga = cv2.resize(img_rgb, (640, 360))
    dlib_rects = detector_hog(img_rgb_vga, 1) # <- 이미지 피라미드 수 변경
    if len(dlib_rects) < 1:
        return img_orig

    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb_vga, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)

#### 코드 수정 후 실행 속도가 현저히 느리진다. 30ms -> 90ms
실시간 구동이 거의 불가능하다.  
#### 실행시간을 줄이는 방법을 찾아보자
hog detector를 deeplearning 기반 detector로 변경 가능  
https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/  
opencv는 intel cpu을 사용 시 dnn모듈이 가속화 지원